# Análise de Sentimento - Reviews Mercado Livre (Dataset 2)

## Objetivo
Este notebook tem como objetivo realizar uma análise exploratória e de sentimento sobre o dataset `reviews_mercadolivre_com_br_2.json`.
Vamos investigar a distribuição das notas, padrões textuais e extrair insights sobre a satisfação dos clientes.

## Dataset
O dataset contém reviews de produtos extraídos do site Mercado Livre, com informações de data, nota (rating), texto do comentário e URL do produto.
O arquivo analisado é o `reviews_mercadolivre_com_br_2.json`.

### Configuração e Importação de Bibliotecas

In [ ]:
# Instalação de bibliotecas necessárias
# !pip install pandas numpy matplotlib seaborn nltk wordcloud

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Configuração de Estilo Visual
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# NLP
import nltk
from wordcloud import WordCloud

# Baixando recursos do NLTK necessários para processamento de texto
recursos_nltk = ['punkt', 'stopwords', 'punkt_tab']
for recurso in recursos_nltk:
    try:
        nltk.data.find(f'tokenizers/{recurso}')
    except LookupError:
        nltk.download(recurso, quiet=True)
        print(f"Recurso '{recurso}' baixado.")